In [172]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import os
import unidecode
import sys, getopt
import csv

In [181]:
#Functions to get the data in a dataframe using BeautifulSoup
def get_tables(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),"html.parser")
    all_tables = soup.findAll("tbody")

    return all_tables

In [99]:
# Fields of interest
similar_players_fields = ["player",  "nationality", "squad", "compare"]
sub_features = ["player_url", "nation_url","squad_url", "comparison_url"]
similar_players_fields = ["nationality"]

In [178]:
# Subsequently, the URL will come from the players datasets
# This is just a sample URL to ensure that we can get all the information for the player
url = "https://fbref.com/en/players/d70ce98e/Lionel-Messi"


In [179]:
players_name = url.split("/")[-1]
print(players_name)

Lionel-Messi


In [180]:
def get_frame_similar_players(table):
    main_player = []
    player_name = []
    player_url = []
    nationality =[]
    nation_url = []
    squad = []
    squad_url = []
    comparison_url = []
    rows_player = table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):

                links = row.findAll("a")
                player_url.append(links[0].get("href"))
                player_name.append(links[0].text)

                nation_url.append(links[1].get("href"))
                nationality.append(links[1].text)

                squad.append(links[2].text)
                squad_url.append(links[2].get("href"))

                comparison_url.append(links[3].get("href"))

                main_player.append(players_name)
                

    df_similar_players = pd.DataFrame({  "main_player":main_player,
                                        "compared_player":player_name, 
                                        "player_url":player_url,
                                        "nationality":nationality,
                                        "nationality_url":nation_url,
                                        "squad":squad,
                                        "squad_url": squad_url,
                                        "comparison_url": comparison_url
                                    })
    return df_similar_players

In [182]:
similar_players_table = get_tables(url)[2]

In [174]:
df_similar_players = get_frame_similar_players(similar_players_table)   

In [ ]:
df_similar_players.head()

In [167]:
try:
    players_directory = os.mkdir("../data/"+players_name+"/")
except Exception as e:
    print(e)


[Errno 17] File exists: '../data/Lionel-Messi/'


In [176]:
df_similar_players.to_csv("../data/"+players_name+"/"+players_name+"-compare.csv", index=False)

In [ ]:
# Possible condition to check to ensure that the currently visited player has a table comparing him with others player will entail checking 
# the number of tables contained on his home page. Less than 14 tables could imply that the user is not been compared to other players. 

In [185]:
similar_forwards_tables = get_tables(url)[3]

In [187]:
df_similar_forwards_tables = get_frame_similar_players(similar_forwards_tables)

In [190]:
df_similar_forwards_tables.to_csv("../data/"+players_name+"/"+players_name+"_forward_player_compare.csv", index=False)

In [191]:
df_similar_forwards_tables.head()

,main_player,compared_player,player_url,nationality,nationality_url,squad,squad_url,comparison_url
0,Lionel-Messi,Karim Benzema,/en/players/70d74ece/Karim-Benzema,fr FRA,/en/country/FRA/France-Football,Real Madrid,/en/squads/53a2f082/Real-Madrid-Stats,/en/stathead/player_comparison.cgi?show_form=1...
1,Lionel-Messi,Kylian Mbappé,/en/players/42fd9c7f/Kylian-Mbappe,fr FRA,/en/country/FRA/France-Football,Paris Saint-Germain,/en/squads/e2d8892c/Paris-Saint-Germain-Stats,/en/stathead/player_comparison.cgi?show_form=1...
2,Lionel-Messi,Memphis Depay,/en/players/8f696594/Memphis-Depay,nl NED,/en/country/NED/Netherlands-Football,Barcelona,/en/squads/206d90db/Barcelona-Stats,/en/stathead/player_comparison.cgi?show_form=1...
3,Lionel-Messi,Cristiano Ronaldo,/en/players/dea698d9/Cristiano-Ronaldo,pt POR,/en/country/POR/Portugal-Football,Manchester United,/en/squads/19538871/Manchester-United-Stats,/en/stathead/player_comparison.cgi?show_form=1...
4,Lionel-Messi,Andrej Kramarić,/en/players/603cb947/Andrej-Kramaric,hr CRO,/en/country/CRO/Croatia-Football,Hoffenheim,/en/squads/033ea6b8/Hoffenheim-Stats,/en/stathead/player_comparison.cgi?show_form=1...
